<a href="https://colab.research.google.com/github/frypan1/data-science/blob/main/cardekho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs


In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [ ]:
base_url = 'https://www.cardekho.com/newcars#brands'
sesh = requests.Session()
resp = sesh.get(base_url,headers=headers)
sp = bs(resp.text,'html.parser')

In [ ]:
def get_brands(soup):
  brands = []
  for i in soup.find_all('div',class_='gsc-ta-content gsc-ta-active'):
    for j in i.find_all('a',href=True,class_='BrIconNewCar'):
        brands.append(j['href'])
  return brands

def get_price(soup):
  price = []
  for i in soup.find_all('li',class_='gsc_col-xs-12 gsc_col-sm-6 gsc_col-md-12 gsc_col-lg-12',id=True):
    price.append(i.find('div',class_='price').text.split('*')[0])
  return price

def get_car(soup):
  cars_list = []
  for i in soup.find_all('li',class_='gsc_col-xs-12 gsc_col-sm-6 gsc_col-md-12 gsc_col-lg-12',id=True):
    cars_list.append(i.find('a',class_='',href=True,title=True).text)
  return cars_list

def other_deets(soup):
  l = []
  for i in soup.find_all('li',class_='gsc_col-xs-12 gsc_col-sm-6 gsc_col-md-12 gsc_col-lg-12',id=True):
    temp = []
    for j in i.find_all('span'):
      if j.text.strip() == 'Rs.' or j.text.strip() == '(Price in New Delhi)' or j.text.strip() == 'w':
        continue
      else:
        temp.append(j.text)
    l.append(temp)
  fuel_type = [i[0] for i in l]
  milage = [i[1] if i[1][0].isdigit() else np.nan for i in l]
  transmission = [i[-2] if i[-2][0].isalpha() else np.nan for i in l]
  return fuel_type,milage,transmission

In [ ]:
def magic():
  data = pd.DataFrame()
  for i in get_brands(sp):
    url = f'https://www.cardekho.com{i}'
    session = requests.Session()
    response = session.get(url,headers=headers)
    soup = bs(response.text,'html.parser')
    new_data = pd.DataFrame({'car':get_car(soup),'price':get_price(soup),'fuel_type':other_deets(soup)[0],'milage':other_deets(soup)[1],'transmission':other_deets(soup)[2]})
    data = pd.concat([data, new_data], ignore_index=True)
  return data

In [ ]:
data = magic()

In [ ]:
data.isna().sum()

car               0
price             0
fuel_type         0
milage          109
transmission      0
dtype: int64

In [ ]:
data.fuel_type.unique()

array(['Petrol', 'Petrol / CNG', 'Diesel / Petrol', 'Diesel',
       'Diesel / Petrol / CNG', 'Electric', 'Diesel / CNG', 'CNG'],
      dtype=object)

In [ ]:
data[data['fuel_type']=='Electric']

,car,price,fuel_type,milage,transmission
29,Tata Punch EV,Rs.10.99 - 15.49 Lakh,Electric,NaN,NaN
30,Tata Nexon EV,Rs.14.49 - 19.49 Lakh,Electric,NaN,NaN
31,Tata Tiago EV,Rs.7.99 - 11.89 Lakh,Electric,NaN,NaN
34,Tata Tigor EV,Rs.12.49 - 13.75 Lakh,Electric,NaN,NaN
39,Kia EV6,Rs.60.97 - 65.97 Lakh,Electric,NaN,NaN
61,Hyundai Kona Electric,Rs.23.84 - 24.03 Lakh,Electric,NaN,NaN
66,Hyundai IONIQ 5,Rs.46.05 Lakh,Electric,NaN,NaN
74,Mahindra XUV400 EV,Rs.15.49 - 19.39 Lakh,Electric,NaN,NaN
86,MG Comet EV,Rs.6.99 - 9.53 Lakh,Electric,NaN,NaN
89,MG ZS EV,Rs.18.98 - 25.20 Lakh,Electric,NaN,NaN


In [ ]:
data.transmission.unique()

array(['Manual/Automatic', 'Manual', 'Automatic', nan], dtype=object)

In [ ]:
data[data['transmission']=='Check Exciting Offers']

,car,price,fuel_type,milage,transmission


In [ ]:
data['transmission'] = np.where(data['transmission']=='Contact Dealer','Automatic',data['transmission'])

In [ ]:
data

,car,price,fuel_type,milage,transmission
0,Maruti Suzuki Swift,Rs.6.49 - 9.59 Lakh,Petrol,24.8 to 25.75 kmpl,Manual/Automatic
1,Maruti Suzuki Brezza,Rs.8.34 - 14.14 Lakh,Petrol / CNG,17.38 to 19.89 kmpl,Manual/Automatic
2,Maruti Suzuki Ertiga,Rs.8.69 - 13.03 Lakh,Petrol / CNG,20.3 to 20.51 kmpl,Manual/Automatic
3,Maruti Suzuki FRONX,Rs.7.51 - 13.04 Lakh,Petrol / CNG,20.01 to 22.89 kmpl,Manual/Automatic
4,Maruti Suzuki Baleno,Rs.6.66 - 9.83 Lakh,Petrol / CNG,22.35 to 22.94 kmpl,Manual/Automatic
...,...,...,...,...,...
240,Volvo XC90,Rs.1.01 Cr,Petrol,17.2 kmpl,Automatic
241,Volvo XC60,Rs.68.90 Lakh,Petrol,11.2 kmpl,Automatic
242,Volvo S90,Rs.68.25 Lakh,Petrol,NaN,Automatic
243,Volvo C40 Recharge,Rs.62.95 Lakh,Electric,NaN,NaN


In [ ]:
data['transmission'] = np.where(data['fuel_type']=='Electric','Automatic',data['transmission'])


In [ ]:
data

,car,price,fuel_type,milage,transmission
0,Maruti Suzuki Swift,Rs.6.49 - 9.59 Lakh,Petrol,24.8 to 25.75 kmpl,Manual/Automatic
1,Maruti Suzuki Brezza,Rs.8.34 - 14.14 Lakh,Petrol / CNG,17.38 to 19.89 kmpl,Manual/Automatic
2,Maruti Suzuki Ertiga,Rs.8.69 - 13.03 Lakh,Petrol / CNG,20.3 to 20.51 kmpl,Manual/Automatic
3,Maruti Suzuki FRONX,Rs.7.51 - 13.04 Lakh,Petrol / CNG,20.01 to 22.89 kmpl,Manual/Automatic
4,Maruti Suzuki Baleno,Rs.6.66 - 9.83 Lakh,Petrol / CNG,22.35 to 22.94 kmpl,Manual/Automatic
...,...,...,...,...,...
240,Volvo XC90,Rs.1.01 Cr,Petrol,17.2 kmpl,Automatic
241,Volvo XC60,Rs.68.90 Lakh,Petrol,11.2 kmpl,Automatic
242,Volvo S90,Rs.68.25 Lakh,Petrol,NaN,Automatic
243,Volvo C40 Recharge,Rs.62.95 Lakh,Electric,NaN,Automatic


In [ ]:

# # Create a session
# session = requests.Session()

# # Define the URL
# url = 'https://www.cardekho.com/toyota-cars'

# # Set up headers with a User-Agent
# headers = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
# }

# # Make the request with headers
# response = session.get(url, headers=headers)
# # response = session.get(url)

# # Check if the request was successful
# if response.status_code == 200:
#     soup = bs(response.text, 'html.parser')
#     # Proceed with parsing the soup object
#     print(soup.prettify())
# else:
#     print(f"Failed to retrieve the page. Status code: {response.status_code}")


In [ ]:
data.to_csv('car_data.csv')